In [1]:
from protocols.ourftsa22.client import Client
from protocols.ourftsa22.server import Server
from protocols.buildingblocks.JoyeLibert import JLS

dimension = 100
inputsize = 16 
keysize = 2048
threshold = 7
nclients = 10
dropout = 3
publicparam, _ , _ = JLS(nclients, inputsize, dimension).generate_keys()

Client.set_scenario(dimension, inputsize, keysize, threshold, nclients, publicparam)
Server.set_scenario(dimension, inputsize, keysize, threshold, nclients, publicparam)


# Create clients and server

In [2]:
clients = {}
for i in range(nclients):
    idx = i+1
    clients[idx] = Client(idx)

server = Server()

### **Setup-Register** phase

In [3]:
# The clients 
allpks = {}
allpkc = {}
for i in range(nclients):
    user, pks, pkc = clients[i+1].setup_register()
    allpks[user] = pks
    allpkc[user] = pkc

In [4]:
# The server
allpks, allpkc = server.setup_register(allpks, allpkc)

### **Setup-KeySetup** phase

In [5]:
# The clients
allekshares = {}
for i in range(nclients):
    user, eshares = clients[i+1].setup_keysetup(allpks, allpkc)
    allekshares[user] = eshares

In [6]:
# The server 
allekshares = server.setup_keysetup(allekshares)

In [7]:
# The clients
for i in range(nclients):
    clients[i+1].setup_keysetup2(allekshares[i+1])

### **Online-Encrypt** phase

In [8]:
# The clients
allebshares = {}
allY = {}
for i in range(nclients):
    clients[i+1].new_fl_step()
    user, eshares, Y = clients[i+1].online_encrypt()
    allebshares[user] = eshares
    allY[user] = Y


In [9]:
# drop some clients
nclientsnew = nclients - dropout
allY = {idx:y for idx, y in allY.items() if idx <= nclientsnew }
allebshares = {idx:y for idx, y in allebshares.items() if idx <= nclientsnew }

In [10]:
# The server
allebshares = server.online_encrypt(allebshares, allY)

### **Online-Construct** phase

In [11]:
# The clients
allbshares = {}
Ybarshares = {}
for i in range(nclientsnew):
    user, bshares, Ybarshare = clients[i+1].online_construct(allebshares[i+1])
    allbshares[user] = bshares 
    Ybarshares[user] = Ybarshare

In [12]:
# The server
YbarsharesValues = list(Ybarshares.values())
if YbarsharesValues.count(None) == len(YbarsharesValues):
    Ybarshares = None

sumX = server.online_construct(allbshares, Ybarshares)

In [13]:
# Verify the results
summ=clients[1].X
from operator import add
for i in range(1, nclientsnew):
    summ = list(map(add, summ, clients[i+1].X))

# print(sumX)
# print(summ)
print("Verify: ", sumX  == summ )


Verify:  True
